[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). Instance produced by Yu Zhou Zhang. All rights reserved.

# My Covid-19 Dashboard

This is my Covid-19 Dashboard, which displays the figures for Covid tests, cases and deaths in England over time, both with raw data and proportionally. The dashboard is displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. 

In [80]:
from IPython.display import clear_output
from time import sleep
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [41]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [43]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open('testsAndCases.json', 'r') as fileinput:
    filedata = fileinput.read()
    jsondata=json.loads(filedata)
print(jsondata)

{'data': [{'date': '2022-11-17', 'tests': None, 'cases': 20866, 'deaths': 696}, {'date': '2022-11-16', 'tests': 39866, 'cases': 0, 'deaths': 0}, {'date': '2022-11-15', 'tests': 48208, 'cases': 0, 'deaths': 0}, {'date': '2022-11-14', 'tests': 55866, 'cases': 0, 'deaths': 0}, {'date': '2022-11-13', 'tests': 33868, 'cases': 0, 'deaths': 0}, {'date': '2022-11-12', 'tests': 30852, 'cases': 0, 'deaths': 0}, {'date': '2022-11-11', 'tests': 45861, 'cases': 0, 'deaths': 0}, {'date': '2022-11-10', 'tests': 47716, 'cases': 21391, 'deaths': 799}, {'date': '2022-11-09', 'tests': 48105, 'cases': 0, 'deaths': 0}, {'date': '2022-11-08', 'tests': 53635, 'cases': 0, 'deaths': 0}, {'date': '2022-11-07', 'tests': 58322, 'cases': 0, 'deaths': 0}, {'date': '2022-11-06', 'tests': 35762, 'cases': 0, 'deaths': 0}, {'date': '2022-11-05', 'tests': 33745, 'cases': 0, 'deaths': 0}, {'date': '2022-11-04', 'tests': 48223, 'cases': 0, 'deaths': 0}, {'date': '2022-11-03', 'tests': 53541, 'cases': 28209, 'deaths': 999}

In [86]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata, proportional=False):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist = rawdata['data']
    
    dates = [record['date'] for record in datalist]
    dates.sort()
    startDate = parse_date(dates[0])
    endDate = parse_date(dates[-1])
    index = pd.date_range(startDate, endDate, freq='D')
    
    myDataFrame = pd.DataFrame(index=index, columns=['tests', 'cases', 'deaths'])
    
    
    maximum_values = {}
    # saving the maximum values of columns
    for dayData in datalist:
        for k, v in dayData.items():
            if k in ['tests', 'cases', 'deaths']:
                
                if v is not None:
                    
                    if v > maximum_values.get(k, 0):
                        maximum_values[k] = v
                        
    
    
    
    for record in datalist:
        date = parse_date(record['date'])
        for column in ['tests', 'cases', 'deaths']:
            
            if pd.isna(myDataFrame.loc[date, column]):
                value = float(record[column]) if record[column] != None else 0.0
                if proportional:
                    value /= maximum_values.get(column, 1)
                
                myDataFrame.loc[date, column] = value
    myDataFrame.fillna(0.0, inplace=True)
    
    
    return myDataFrame

# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata)# df is the dataframe for plotting
dfProportional=wrangle_data(jsondata, proportional=True)


Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [93]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
    'areaType=nation',
    'areaName=England'
]
    structure = {
    "date": "date",
    "tests": "newTestsByPublishDate",
    "cases": "newCasesByPublishDate",
    "deaths": "newDeaths28DaysByPublishDate"
}
    api = Cov19API(filters=filters, structure=structure)
    testsAndCases=api.get_json()
    return testsAndCases # return data read from the API

In [95]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    try:
        apidata=access_api()
        global df
        global dfProportional
        df=wrangle_data(apidata)
        dfProportional=wrangle_data(apidata, proportional=True)
        apibutton.icon="check"
        apibutton.disabled=True
    
    # wrangle the data and overwrite the dataframe for plotting
    
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    #refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    except:
        print('Sorry, it seems there is a problem fetching the data')
        apibutton.icon="unlink"
        apibutton.description='Unavailable'
        

    
apibutton=wdg.Button(
    description='Fetch Data', # you may want to change this...
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets
display(apibutton)

# run all cells before clicking on this button

Button(description='Fetch Data', icon='download', style=ButtonStyle(), tooltip='Keep calm and carry on')

# Comparing tests, cases and deaths

Below, you can select the metrics that you want to view (Ctrl + click to select multiple) and choose whether to view raw figures or figures scaled proportionally to its maximum value.
This proportional scaling allows you to visualise more clearly how each metric is influenced by others.

In [92]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

testsCasesDeathsColumns = wdg.SelectMultiple(
    options = ['tests', 'cases', 'deaths'],
    value = ['cases', 'deaths'],
    rows = 3,
    description = 'Parameters',
    disabled = False)

proportion=wdg.RadioButtons(
    options=['raw', 'proportional'],
    value='proportional',
    layout={'width': 'max-content'},
    description='Scale:',
    disabled=False)
    
controls=wdg.HBox([testsCasesDeathsColumns, proportion])

def testsCasesDeathsGraph(columns, proportion):
    
    numberOfColumns = len(columns)
    if numberOfColumns > 0:
        if proportion == 'proportional':
            dfProportional.plot(y = list(columns))
        else:
            df.plot(y = list(columns))
        plt.show()
    else:
        print('Click to select data for graph')
output = wdg.interactive_output(testsCasesDeathsGraph, {'columns': testsCasesDeathsColumns, 'proportion': proportion})

form=wdg.HBox([output, controls])
display(form)


## Acknowledgements

Created under the guidance of Prof. Fabrizio Smeraldi. Thanks for the support!

*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*